# Import

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score

from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

df = pd.read_csv("data/food_crises_interpol.csv") # Read data into DataFrame
news_df = pd.read_csv("data/articles_topics_positivity.csv") # Read news data into DataFrame

# Processing Data

In [3]:
df["date"] = pd.to_datetime(df["year_month"], format="%Y-%m") # Create date column
print(len(df['district'].unique()))
df.set_index(["date", "county"], inplace=True) # Set index

77


In [5]:
news_df["date"] = pd.to_datetime(
    pd.to_datetime(news_df["date"], format="%Y-%m-%d").dt.strftime("%Y-%m"),
    format="%Y-%m",
)

In [6]:
def create_news_features(columns):
    cols = []
    for column in columns:
        col = news_df.groupby(["date", "county"])[column].mean()
        col = col.fillna(0)
        # col = col.rolling(3).mean()
        # col = col.shift(3)
        cols.append(col)
    return pd.concat(cols, axis=1)

In [7]:
news_features = create_news_features(['hunger_positive', 'hunger_negative', 'refugees_positive', 'refugees_negative', 'humanitarian_positive', 'humanitarian_negative', 'conflict_positive', 'conflict_negative', 'vulnerability_positive', 'vulnerability_negative'])

# Merge

In [8]:
df.sort_index(level=0, inplace=True) # Sort DataFrame by date

In [9]:
df

Unnamed: 0      district                   region  \
date       county                                                            
2009-11-01 Abiemnhom           7940     Abiemnhom                    Unity   
           Akobo               5124         Akobo                  Jonglei   
           Aweil Centre        6916  Aweil Center  Northern Bahr el Ghazal   
           Aweil East          8580    Aweil East  Northern Bahr el Ghazal   
           Aweil North          260   Aweil North  Northern Bahr el Ghazal   
...                             ...           ...                      ...   
2020-01-01 Wulu                4094          Wulu                    Lakes   
           Yambio              1150        Yambio        Western Equatoria   
           Yei                 1022           Yei        Central Equatoria   
           Yirol East          4350    Yirol East                    Lakes   
           Yirol West          3582    Yirol West                    Lakes   

                         ipc  year_month  ndvi_mean  ndvi_anom  rain_mean  \
date       county                                                           
2009-11-01 Abiemnhom     2.0  2009-11-01   0.452458   97.17758   1.085453   
           Akobo         3.0  2009-11-01   0.450313   79.39274   1.285720   
           Aweil Centre  1.0  2009-11-01   0.494648  101.18030   5.601856   
           Aweil East    2.0  2009-11-01   0.418445   97.18519   1.084202   
           Aweil North   1.0  2009-11-01   0.422307   98.88503   1.596338   
...                      ...         ...        ...        ...        ...   
2020-01-01 Wulu          2.0  2020-01-01   0.390809  115.31650   0.216196   
           Yambio        3.0  2020-01-01   0.480063  113.04510   0.609318   
           Yei           3.0  2020-01-01   0.526050  124.30660   1.210738   
           Yirol East    3.0  2020-01-01   0.466752  115.83420   0.193205   
           Yirol West    3.0  2020-01-01   0.395880  125.25050   0.345029   

                         rain_anom    et_mean  ...  sum_fatalities  \
date       county                              ...                   
2009-11-01 Abiemnhom      0.415071   1.202695  ...               0   
           Akobo         -2.052032   5.713924  ...               0   
           Aweil Centre   3.108650   2.968859  ...               0   
           Aweil East     0.599545   1.310700  ...               0   
           Aweil North    0.852844   1.387875  ...               0   
...                            ...        ...  ...             ...   
2020-01-01 Wulu          -0.099790   1.810135  ...               1   
           Yambio        -0.765759   6.120980  ...               1   
           Yei           -0.921322  11.679240  ...               1   
           Yirol East    -0.015236   3.880718  ...               0   
           Yirol West    -0.057713   2.574695  ...               0   

                         food_price_idx       area  cropland_pct        pop  \
date       county                                                             
2009-11-01 Abiemnhom           0.943680   2408.233      1.742261   20702.00   
           Akobo               1.049001   9027.626     10.046920  158222.00   
           Aweil Centre        0.908136  11202.960      0.173833   48376.00   
           Aweil East          0.932502   6400.981     26.804220  345977.00   
           Aweil North         0.875988   6369.012     22.906570  146204.00   
...                                 ...        ...           ...        ...   
2020-01-01 Wulu               80.615710  11894.780      0.032678   45158.50   
           Yambio             74.486710   8896.165     19.126490  197278.90   
           Yei                90.145640   6713.251     16.303490  288875.40   
           Yirol East         99.849660   5599.893      2.482445   73693.13   
           Yirol West         98.732880   4879.717      0.462767  111980.60   

                         ruggedness_mean  pasture_pct  prev_ipc  2pre

In [10]:
df = df.join(news_features, how="left") # Join df with created news features

In [11]:
# Replace the NaN values in specific columns with 0
columns_to_fill = ['hunger_positive', 'hunger_negative', 'refugees_positive', 'refugees_negative', 'conflict_positive', 'conflict_negative', 'humanitarian_positive', 'humanitarian_negative', 'vulnerability_positive', 'vulnerability_negative']
df[columns_to_fill] = df[columns_to_fill].fillna(0)


In [12]:
df

Unnamed: 0      district                   region  \
date       county                                                            
2009-11-01 Abiemnhom           7940     Abiemnhom                    Unity   
           Akobo               5124         Akobo                  Jonglei   
           Aweil Centre        6916  Aweil Center  Northern Bahr el Ghazal   
           Aweil East          8580    Aweil East  Northern Bahr el Ghazal   
           Aweil North          260   Aweil North  Northern Bahr el Ghazal   
...                             ...           ...                      ...   
2020-01-01 Wulu                4094          Wulu                    Lakes   
           Yambio              1150        Yambio        Western Equatoria   
           Yei                 1022           Yei        Central Equatoria   
           Yirol East          4350    Yirol East                    Lakes   
           Yirol West          3582    Yirol West                    Lakes   

                         ipc  year_month  ndvi_mean  ndvi_anom  rain_mean  \
date       county                                                           
2009-11-01 Abiemnhom     2.0  2009-11-01   0.452458   97.17758   1.085453   
           Akobo         3.0  2009-11-01   0.450313   79.39274   1.285720   
           Aweil Centre  1.0  2009-11-01   0.494648  101.18030   5.601856   
           Aweil East    2.0  2009-11-01   0.418445   97.18519   1.084202   
           Aweil North   1.0  2009-11-01   0.422307   98.88503   1.596338   
...                      ...         ...        ...        ...        ...   
2020-01-01 Wulu          2.0  2020-01-01   0.390809  115.31650   0.216196   
           Yambio        3.0  2020-01-01   0.480063  113.04510   0.609318   
           Yei           3.0  2020-01-01   0.526050  124.30660   1.210738   
           Yirol East    3.0  2020-01-01   0.466752  115.83420   0.193205   
           Yirol West    3.0  2020-01-01   0.395880  125.25050   0.345029   

                         rain_anom    et_mean  ...  hunger_positive  \
date       county                              ...                    
2009-11-01 Abiemnhom      0.415071   1.202695  ...              0.0   
           Akobo         -2.052032   5.713924  ...              0.0   
           Aweil Centre   3.108650   2.968859  ...              0.0   
           Aweil East     0.599545   1.310700  ...              0.0   
           Aweil North    0.852844   1.387875  ...              0.0   
...                            ...        ...  ...              ...   
2020-01-01 Wulu          -0.099790   1.810135  ...              0.0   
           Yambio        -0.765759   6.120980  ...              0.0   
           Yei           -0.921322  11.679240  ...              0.0   
           Yirol East    -0.015236   3.880718  ...              0.0   
           Yirol West    -0.057713   2.574695  ...              0.0   

                         hunger_negative  refugees_positive  \
date       county                                             
2009-11-01 Abiemnhom                 0.0                0.0   
           Akobo                     0.0                0.0   
           Aweil Centre              0.0                0.0   
           Aweil East                0.0                0.0   
           Aweil North               0.0                0.0   
...                                  ...                ...   
2020-01-01 Wulu                      0.0                0.0   
           Yambio                    0.0                0.0   
           Yei                       0.0                0.0   
           Yirol East                0.0                0.0   
           Yirol West                0.0                0.0   

                         refugees_negative  humanitarian_positive  \
date       county                                                   
2009-11-01 Abiemnhom                   0.0                    0.0   
           Akobo                       0.0               

In [13]:
df.to_csv("data/food_crises_interpol_news.csv")